# インポート

In [0]:
!pip install japanize_matplotlib
!pip install optuna
!pip install pandas-profiling
!pip install feather-format
!pip install category_encoders
!pip install catboost

    100% |████████████████████████████████| 4.1MB 9.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/47/d1/ba/1a686af7cc042edde2c2f4cb18bd981f7eefdcbfe688590d25
Successfully built japanize-matplotlib
    100% |████████████████████████████████| 71kB 2.8MB/s 
    100% |████████████████████████████████| 81kB 6.2MB/s 
    100% |████████████████████████████████| 51kB 22.2MB/s 
    100% |████████████████████████████████| 92kB 28.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/6d/b5/e7e634d0d7d9f93bfa11bc65a0b6cfeb1fa17ba948f6eff2ae
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna pyperclip
    100% |████████████████████████████████| 12.4MB 3.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
Successfully built feather-format
    100% |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
import numpy as np
import pandas as pd
import pandas.tools.plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pylab
import matplotlib.gridspec
import japanize_matplotlib
import math
import seaborn as sns
import sys
import scipy
import IPython
import warnings
import collections
import random
import time
import optuna
import sklearn
import sklearn.preprocessing
from sklearn.ensemble import RandomForestRegressor
import sklearn.discriminant_analysis 
import sklearn.preprocessing
import sklearn.manifold
import sklearn.linear_model
import sklearn.neighbors
import sklearn.utils
import sklearn.tree 
import sklearn.neural_network
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,log_loss
from sklearn.model_selection import cross_val_score, GridSearchCV,KFold
from sklearn.externals import joblib
import chainer
import chainer.training 
import xgboost
import lightgbm
import pandas_profiling as pdp
import feather
from sklearn.ensemble import IsolationForest
import catboost

# Google Driveと連携

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive/my-drive
!google-drive-ocamlfuse drive/my-drive 

# マイモジュール インポート

In [0]:
import os
os.chdir("drive/my-drive/colab/house_price/")

In [0]:
from models import lgbm
from function import model_train, submit, data, importance, selection
from train_predict import train
import importlib

In [0]:
importlib.reload(data)
importlib.reload(model_train)
importlib.reload(train)
importlib.reload(lgbm)
importlib.reload(submit)


# 前処理

## trainとtestのfeatherファイル作成

In [0]:
!python scripts/convert_to_feather.py

## カテゴリ変数削除・欠損値埋め

In [0]:
!python features/preprocessing.py

## 基本特徴量追加(欠損・ゼロフラグ・One-hot)

In [0]:
!python features/create.py

(1460, 81)

# 可視化

In [0]:
df_train, df_test = data.get_origin_dataframe()

## Pandas Profile

In [0]:
pdp.ProfileReport(df_train)

# 色々見てみる

In [0]:
importance = joblib.load("features/importances/base_line+delete_outliers+log+delete_importance_0")

In [75]:
importance.sort_values("importance",ascending=False).head(20)

importance
GrLivArea      126.666667
OverallQual    116.333333
LotArea        115.666667
OverallCond     78.666667
TotalBsmtSF     73.000000
YearBuilt       68.000000
GarageArea      61.333333
BsmtFinSF1      59.333333
LotFrontage     51.000000
GarageYrBlt     49.000000
BsmtUnfSF       48.666667
OpenPorchSF     41.000000
YearRemodAdd    40.000000
1stFlrSF        39.333333
WoodDeckSF      34.000000
EnclosedPorch   29.666667
MoSold          29.333333
YrSold          28.000000
2ndFlrSF        27.666667
ScreenPorch     25.333333

In [0]:
corr_to_target = joblib.load("data/input/target_and_train_origin_corr")

In [77]:
corr_to_target.head(10)

corr_to_target
OverallQual         0.790982
GrLivArea           0.708624
GarageCars          0.640409
GarageArea          0.623431
TotalBsmtSF         0.613581
1stFlrSF            0.605852
FullBath            0.560664
TotRmsAbvGrd        0.533723
YearBuilt           0.522897
YearRemodAdd        0.507101